# Color Code State Injection Circuit

Fig. 1a from QuEra MSD paper, recreated using `bloqade.squin`.

- Qubits 0–5: |0⟩
- Qubit 6: |Ψ(θ,φ)⟩ (for Clifford sim, all start |0⟩)

**Structure (6 columns):**
- Column 1: √Y† on qubits 0–5
- Column 2: CNOT(0,1), CNOT(0,2), CNOT(3,4), CNOT(3,5), CNOT(6,4)
- Column 3: CNOT(0,3), CNOT(1,2), CNOT(4,5), CNOT(6,5)
- Column 4: √Y on qubits 1,2,3,4,5,6
- Column 5: CNOT(0,1), CNOT(2,3), CNOT(4,5), CNOT(6,3)
- Column 6: √Y on qubits 2,4,5,6

In [1]:
from bloqade import squin
import bloqade.stim
import bloqade.tsim
import subprocess
from pathlib import Path
from IPython.display import display

In [2]:
@squin.kernel
def sqrt_y(q):
    """√Y = (I + iY)/√2. Clifford approx: S · H · S† (S† = S³)."""
    squin.s(q)
    squin.h(q)
    squin.s(q)
    squin.s(q)
    squin.s(q)


@squin.kernel
def sqrt_y_dagger(q):
    """√Y† = (I - iY)/√2. Clifford approx: S† · H · S (S† = S³)."""
    squin.s(q)
    squin.s(q)
    squin.s(q)  # S† = S³
    squin.h(q)
    squin.s(q)


@squin.kernel
def colorcode_state_injection():
    """Color code state injection circuit (exact match to Fig. 1a)."""
    q = squin.qalloc(7)

    # Column 1: √Y† on qubits 0-5
    for i in range(6):
        sqrt_y_dagger(q[i])

    # Column 2: CNOTs
    squin.cx(q[0], q[1])
    squin.cx(q[0], q[2])
    squin.cx(q[3], q[4])
    squin.cx(q[3], q[5])
    squin.cx(q[6], q[4])

    # Column 3: CNOTs
    squin.cx(q[0], q[3])
    squin.cx(q[1], q[2])
    squin.cx(q[4], q[5])
    squin.cx(q[6], q[5])

    # Column 4: √Y on qubits 1,2,3,4,5,6
    for i in range(1, 7):
        sqrt_y(q[i])

    # Column 5: CNOTs
    squin.cx(q[0], q[1])
    squin.cx(q[2], q[3])
    squin.cx(q[4], q[5])
    squin.cx(q[6], q[3])

    # Column 6: √Y on qubits 2,4,5,6
    sqrt_y(q[2])
    sqrt_y(q[4])
    sqrt_y(q[5])
    sqrt_y(q[6])

    for i in range(7):
        squin.measure(q[i])

In [3]:
# Build circuit and show diagram in notebook
tsim_circ = bloqade.tsim.Circuit(colorcode_state_injection)
diagram = tsim_circ.diagram(height=400)
display(diagram)

In [6]:
# Save circuit diagram as PNG
out_dir = Path(".")
svg_str = diagram._svg
start = svg_str.find("<svg")
end = svg_str.rfind("</svg>")
svg_content = svg_str[start : end + len("</svg>")]

svg_path = out_dir / "colorcode_circuit.svg"
png_path = out_dir / "colorcode_circuit.png"
svg_path.write_text(svg_content)

subprocess.run(["rsvg-convert", "-o", str(png_path), str(svg_path)], check=True)
print(f"Saved: {png_path.absolute()}")

Saved: /Users/roy-abhishek/VSCode/iQuHack2026/2026-QuEra-Technical-CUDAstudiedmore/roy/colorcode_circuit.png


In [5]:
# Sample with Stim
stim_circ = bloqade.stim.Circuit(colorcode_state_injection)
sampler = stim_circ.compile_sampler()
samples = sampler.sample(shots=1000)
print(f"Samples shape: {samples.shape}")
print("First 5 outcomes:", [''.join(map(str, s.astype(int))) for s in samples[:5]])

Samples shape: (1000, 7)
First 5 outcomes: ['0100110', '0110001', '1001011', '0010110', '0110100']
